In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
%pip install statsmodels --upgrade

You should consider upgrading via the '/Users/enrico/Documents/Faculdade/TCC Local/Projeto/.venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [8]:
from methods.ar import ar, arima
from methods.naive import average_forecast, all_true, all_false
from methods.lstm import lstm, stacked_lstm
from methods.mlp import mlp
from methods.mlp import mlp_keras
from methods.cnn import cnn
from methods.classifiers import knn, gaussian, random_forest
from methods.runner import Runner
import functools


In [9]:
# ar -> n_steps = 2, 4, 8
# arima -> n_steps = 2, 4, 8; order = 2, 4, 8; differencing = 1, 2, 3
# KNN -> n_steps = 2, 4, 8; n_neighbours = 2, 4, 8; distances = 1, 2, 3, 4, 5; 
# MLP -> n_steps = 2, 4, 8; epochs = 50, 100, 200; layers = 1, 2, 3; cells = 50, 100, 200
# LSTM -> n_steps = 2, 4, 8; epochs = 50, 100, 200; layers = 1, 2, 3; cells = 50, 100, 200

models = {}
epochs = [50, 100, 200]
layers = [1, 2, 3]
cells = [50, 100, 200]
orders = [2, 4, 8]
differencing = [1, 2, 3]
n_neighbours = [2, 4, 8]
distances = [1, 2, 3, 4, 5]

for order in orders:
	for diff in differencing:
		models[f"ARIMA_{order}_{diff}"] = functools.partial(arima, order=order, differencing=diff)

for n in n_neighbours:
	for d in distances:
		models[f"KNN_{n}_{d}"] = functools.partial(knn, n_neighbors=n, distance=d)

for epoch in epochs:
	for layer in layers:
		for cell in cells:
			models[f"MLP_{epoch}_{layer}_{cell}"] = functools.partial(mlp_keras, epochs=epoch, layers=layer, cells=cell)

for epoch in epochs:
	for layer in layers:
		for cell in cells:
			models[f"LSTM_{epoch}_{layer}_{cell}"] = functools.partial(lstm, epochs=epoch, layers=layer, cells=cell)

In [5]:
# models = {
# 	"Naive": average_forecast,
# 	"All True": all_true,
# 	"All False": all_false,
# 	# "AR": ar,
# 	"ARIMA": arima,
# 	# "MLP": mlp,
# 	# "KNN": knn,
# 	# "Gaussian": gaussian,
# 	# "RandomForest": random_forest,
# 	# "MLP Keras": mlp_keras,
# 	# "CNN": cnn,
# 	# "LSTM": lstm,
# 	# "Stacked LSTM": stacked_lstm
# }

In [12]:
runner = Runner(models)
# runner.run("accuracy")
runner.continue_run("202204211108", "accuracy")

Running only for datasets with more then 240.0 rows
